In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)

sales = pd.read_csv("C:/Users/emil_/Desktop/ITAM_Maestrtía/GranEscala/Tarea01/competitive-data-science-predict-future-sales/sales_train.csv")
items = pd.read_csv("C:/Users/emil_/Desktop/ITAM_Maestrtía/GranEscala/Tarea01/competitive-data-science-predict-future-sales/items.csv")
shops = pd.read_csv("C:/Users/emil_/Desktop/ITAM_Maestrtía/GranEscala/Tarea01/competitive-data-science-predict-future-sales/shops.csv")
item_categories = pd.read_csv("C:/Users/emil_/Desktop/ITAM_Maestrtía/GranEscala/Tarea01/competitive-data-science-predict-future-sales/item_categories.csv")
test = pd.read_csv("C:/Users/emil_/Desktop/ITAM_Maestrtía/GranEscala/Tarea01/competitive-data-science-predict-future-sales/test.csv")


In [2]:
sales["date"] = pd.to_datetime(sales["date"], format="%d.%m.%Y")
sales["year"] = sales["date"].dt.year
sales["month"] = sales["date"].dt.month

# indice mensual 34 meses 
sales["date_block_num"] = sales["date"].dt.to_period("M").factorize()[0]
#Convertimos la fecha a fecha, y el respectivo mes comenzando la cuenta en cero como se nos pidió en el ejercicio 

In [3]:
#Ahora vamos a pasar de cantidad de producto diaro y cantidad de producto mensual 
monthly = (
    sales
    .groupby(["date_block_num", "shop_id", "item_id"], as_index=False)
    .agg(item_cnt_month=("item_cnt_day", "sum"))
)

# también haremos un clipping
monthly["item_cnt_month"] = monthly["item_cnt_month"].clip(0, 20)


In [4]:
#Hicimos el clipping en 20 porque hay unos pocos outliers muy grandes que distorsionan y dominan el entrenamiento. La 
#mayoría de las observaciones están entre 0 y 5 y una parte muy pequeña llega a 20, el resto sobra por así decirlo. Más cuando 
#Al menos el 75% de las combinaciones producto–tienda–mes venden 1 unidad como máximo

In [5]:
grid = []
for block in monthly["date_block_num"].unique():
    cur = monthly[monthly["date_block_num"] == block]
    shops_in_month = cur["shop_id"].unique()
    items_in_month = cur["item_id"].unique()
    grid.append(
        pd.DataFrame(
            [(block, s, i) for s in shops_in_month for i in items_in_month],
            columns=["date_block_num", "shop_id", "item_id"]
        )
    )

matrix = pd.concat(grid, ignore_index=True)

matrix = matrix.merge(monthly, on=["date_block_num", "shop_id", "item_id"], how="left")
matrix["item_cnt_month"] = matrix["item_cnt_month"].fillna(0)
#si un producto no aparece en un mes, en realidad vendió 0, pero no hay fila.
#Esto crea filas para poder aprender 0 ventas también, esto para completar el modelo y que sea preciso 

In [6]:
items_small = items[["item_id", "item_category_id"]]
matrix = matrix.merge(items_small, on="item_id", how="left")
#Ahora le agregamos la columna de caterogría del producto en cuestión , es para ayudar al modelo a reconocer que los productos de esta categoría se comportan así

In [7]:
#Además sino el modelo memoriza item_id no aprende patrones de categorías

In [8]:
matrix["month"] = (matrix["date_block_num"] % 12).astype(np.int8)
matrix["year"] = (matrix["date_block_num"] // 12).astype(np.int8)
#Nuestro modelo no sabe qué mes del año es ni en qué año va, solo ve un número creciente, por eso le calculamos el residuo para que esos meses 
#se parezcan entre sí. También calcula el año relativo (primer segundo año y así)

In [9]:
def add_lag(df, lags, col):
    tmp = df[["date_block_num", "shop_id", "item_id", col]]
    for lag in lags:
        shifted = tmp.copy()
        shifted["date_block_num"] += lag
        shifted = shifted.rename(columns={col: f"{col}_lag_{lag}"})
        df = df.merge(shifted, on=["date_block_num", "shop_id", "item_id"], how="left")
    return df

matrix = add_lag(matrix, lags=[1,2,3,6,12], col="item_cnt_month")

lag_cols = [c for c in matrix.columns if "lag_" in c]
matrix[lag_cols] = matrix[lag_cols].fillna(0)
#Le hacemos recordar al modelo las ventas anteriores en otra columna(3 es trimestral, 6 es ventas de hace medio año)
matrix.head()


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,month,year,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0,0,32,6.0,40,0,0,0.0,0.0,0.0,0.0,0.0
1,0,0,33,3.0,37,0,0,0.0,0.0,0.0,0.0,0.0
2,0,0,35,1.0,40,0,0,0.0,0.0,0.0,0.0,0.0
3,0,0,43,1.0,40,0,0,0.0,0.0,0.0,0.0,0.0
4,0,0,51,2.0,57,0,0,0.0,0.0,0.0,0.0,0.0


In [10]:
shop_month = (
    matrix.groupby(["date_block_num", "shop_id"], as_index=False)
    .agg(shop_cnt_month=("item_cnt_month", "mean"))
)
matrix = matrix.merge(shop_month, on=["date_block_num", "shop_id"], how="left")
matrix = add_lag(matrix, [1,2,3,6,12], "shop_cnt_month")
matrix.drop(columns=["shop_cnt_month"], inplace=True)
matrix.head()


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,month,year,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,shop_cnt_month_lag_1,shop_cnt_month_lag_2,shop_cnt_month_lag_3,shop_cnt_month_lag_6,shop_cnt_month_lag_12
0,0,0,32,6.0,40,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
1,0,0,33,3.0,37,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
2,0,0,35,1.0,40,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
3,0,0,43,1.0,40,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
4,0,0,51,2.0,57,0,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN


In [11]:
#si el shop_cnt_month_lag_1	es de 0.3, En ese mes, en esa tienda, cada producto vendió en promedio 0.3 unidadess. .

In [12]:
item_month = (
    matrix.groupby(["date_block_num", "item_id"], as_index=False)
    .agg(item_cnt_month_mean=("item_cnt_month", "mean"))
)
matrix = matrix.merge(item_month, on=["date_block_num", "item_id"], how="left")
matrix = add_lag(matrix, [1,2,3,6,12], "item_cnt_month_mean")
matrix.drop(columns=["item_cnt_month_mean"], inplace=True)

matrix = matrix.fillna(0)
matrix.head()

,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,month,year,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12,shop_cnt_month_lag_1,shop_cnt_month_lag_2,shop_cnt_month_lag_3,shop_cnt_month_lag_6,shop_cnt_month_lag_12,item_cnt_month_mean_lag_1,item_cnt_month_mean_lag_2,item_cnt_month_mean_lag_3,item_cnt_month_mean_lag_6,item_cnt_month_mean_lag_12
0,0,0,32,6.0,40,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,33,3.0,37,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,35,1.0,40,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,43,1.0,40,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,51,2.0,57,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
#el promedio de ventas de ese producto (item) a través de todas las tiendas, en un mes pasado.

In [14]:
test["date_block_num"] = 34
test_matrix = test.merge(items_small, on="item_id", how="left")

test_matrix["month"] = (test_matrix["date_block_num"] % 12).astype(np.int8)
test_matrix["year"] = (test_matrix["date_block_num"] // 12).astype(np.int8)

# placeholder del target
test_matrix["item_cnt_month"] = 0
test_matrix.head()

,ID,shop_id,item_id,date_block_num,item_category_id,month,year,item_cnt_month
0,0,5,5037,34,19,10,2,0
1,1,5,5320,34,55,10,2,0
2,2,5,5233,34,19,10,2,0
3,3,5,5232,34,23,10,2,0
4,4,5,5268,34,20,10,2,0


In [15]:
# Construimos el dataset del test con la mismaestructura de features que el train de modelado, preparamos item_cnt_month para el test. 

In [16]:
common_cols = ["date_block_num","shop_id","item_id","item_category_id","month","year","item_cnt_month"]
all_data = pd.concat([matrix[common_cols], test_matrix[common_cols]], ignore_index=True)

all_data = add_lag(all_data, [1,2,3,6,12], "item_cnt_month")
lag_cols = [c for c in all_data.columns if "lag_" in c]
all_data[lag_cols] = all_data[lag_cols].fillna(0)

train_data = all_data[all_data["date_block_num"] <= 33].copy()
test_data  = all_data[all_data["date_block_num"] == 34].copy()


In [17]:
#Ahora hacemos el split
X_train = train_data[train_data["date_block_num"] < 33].drop(columns=["item_cnt_month"])
y_train = train_data[train_data["date_block_num"] < 33]["item_cnt_month"]

X_valid = train_data[train_data["date_block_num"] == 33].drop(columns=["item_cnt_month"])
y_valid = train_data[train_data["date_block_num"] == 33]["item_cnt_month"]

X_test  = test_data.drop(columns=["item_cnt_month"])
#Este split preveien que usemos fechas del futuro con fechas del pasado. 


In [18]:
type(X_train)

pandas.core.frame.DataFrame

In [19]:
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\emil_\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
import os


os.makedirs("../data/processed", exist_ok=True)

X_train.to_csv("../data/processed/X_train.csv", index=False)
y_train.to_frame("item_cnt_month").to_csv("../data/processed/y_train.csv", index=False)

X_valid.to_csv("../data/processed/X_valid.csv", index=False)
y_valid.to_frame("item_cnt_month").to_csv("../data/processed/y_valid.csv", index=False)

X_test.to_csv("../data/processed/X_test.csv", index=False)


In [25]:
import os

os.makedirs("data/prep", exist_ok=True)

X_train.to_csv("data/prep/X_train.csv", index=False)
y_train.to_frame("item_cnt_month").to_csv("data/prep/y_train.csv", index=False)

X_valid.to_csv("data/prep/X_valid.csv", index=False)
y_valid.to_frame("item_cnt_month").to_csv("data/prep/y_valid.csv", index=False)

X_test.to_csv("data/prep/X_test.csv", index=False)

print("Listo: CSV creados en data/prep/")


Listo: CSV creados en data/prep/


In [ ]:
import os
os.getcwd()
